# Make BoW, TFIDF, Lsi

In [65]:
from tfidf_pm import *
pubmed_corpus = PubmedCorpus()

In [66]:
pubmed_corpus.load_corpus()

In [67]:
pubmed_corpus.dictionary.doc2bow(['yo', 'yellow', 'circumference', 'obesity'])

[(49, 1), (561, 1)]

In [ ]:
pubmed_tfidf = models.TfidfModel(pubmed_corpus, normalize=True)
pubmed_tfidf.save(os.path.join(SAVE_LOCATION, 'pubmed_tfidf'))

pubmed_corpus_tfidf = pubmed_tfidf[pubmed_corpus]
pubmed_corpus_tfidf.save(os.path.join(SAVE_LOCATION, 'pubmed_corpus_tfidf'))

pubmed_lsi = models.LsiModel(pubmed_corpus_tfidf, 
                             id2word=pubmed_corpus.dictionary, 
                             num_topics=200)
pubmed_corpus_lsi = pubmed_lsi[pubmed_corpus_tfidf]
pubmed_lsi.save(os.path.join(SAVE_LOCATION, 'pubmed_lsi'))

##### save dictionary

In [68]:
corpora.MmCorpus.serialize('./pmc_models_serialized/pubmed_corpus.mm', pubmed_corpus)

### Folding in a new document and loading models

tfidf = models.tfidfmodel.TfidfModel.load('tfidf')

lsi = models.LsiModel.load('lsi')

Create an index transformer that calculates similarity based on our space
 
test_index1 = similarities.MatrixSimilarity(test_lsi1)

In [69]:
pubmed_corpus = PubmedCorpus()

In [75]:
something = corpora.mmcorpus.MmCorpus('./pmc_models_serialized/pubmed_corpus.mm')

In [ ]:
something.load

In [71]:
??pubmed_corpus.dictionary.load

In [33]:
pubmed_corpus.dictionary.doc2bow(['yo', 'yellow', 'circumference', 'obesity'])

[]

In [72]:
??corpora.MmCorpus

In [51]:
??models.TfidfModel

In [53]:
??corpora.TextCorpus

In [54]:
??interfaces.CorpusABC

In [ ]:
pubmed_corpus.dictionary.

In [ ]:
dictionary = corpora.Dictionary()

In [ ]:
dictionary.doc2bow(['hello', 'something'])

In [ ]:
class thing:
    def __init__(self):
        self.array = [(1, 2, 3), (3,3, 3) , (2, 2, 2)]
    
    def __iter__(self):
        for i, j, k in self.array:
            yield i * 2
            
    def load_stuff(self):
        for i in self:
            print i
            
    def again(self):
        for i in self:
            print i

In [ ]:
a = thing()

In [ ]:
a.load_stuff()

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
    
#     def get_texts(self):
#         directory = os.walk(self.input)
#         for root, dirs, files in directory:
#             for file_name in files:
#                 file_path = os.path.join(root, file_name)
#                 with open(file_path, 'rb') as f:
#                     file_string = to_unicode_or_bust(f.read().lower())
#                     file_tokenized = word_tokenize(file_string)
#                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                     yield file_no_stops

# class PubmedCorpus(corpora.TextCorpus):
        
#     def get_texts(self):
#         for file_path in self.input:
#             with open(file_path, 'rb') as f:
#                 file_string = to_unicode_or_bust(f.read().lower())
#                 file_tokenized = word_tokenize(file_string)
#                 file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                 yield file_no_stops
                    
                
# def load_article_paths():
#     for root, dirs, files in os.walk('./pmc_data/pmc_text_files/'):
#         for name in files:
#             ARTICLE_FILE_PATHS.append(os.path.join(root, name))
#     genre_folders_left = len(GENRE_FOLDERS)
#     completed_genre_folders = 0 
#     for genre_folder in GENRE_FOLDERS:
#         completed_genre_folders += 1
#         genre_folder_path = os.path.join(DATA_PATH, genre_folder)
#         genre_file_list = os.listdir(genre_folder_path)
#         for article_file_title in genre_file_list:
#             article_file_path = os.path.join(genre_folder_path, article_file_title)
#             if os.path.isfile(article_file_path):
#                 ARTICLE_FILE_TITLES.append(article_file_title)
#                 ARTICLE_FILE_PATHS.append(article_file_path)
# #                 with open(article_file_path, 'rb') as f:
# #                     document = f.read()
# #                     ARTICLE_DOCUMENT_LIST.append(document)

#                 print "done with: ", article_file_title
#                 print "progress: ", completed_genre_folders / float(genre_folders_left)
#             else:
#                 sub_article_folder_list = os.listdir(article_file_path)
#                 for sub_article_file_title in sub_article_folder_list:
#                     sub_article_file_path = os.path.join(article_file_path, 
#                             sub_article_file_title)
#                     ARTICLE_FILE_TITLES.append(sub_article_file_title)
#                     ARTICLE_FILE_PATHS.append(sub_article_file_path)
# #                     with open(sub_article_file_path, 'rb') as f:
# #                         document = f.read()
# #                         ARTICLE_DOCUMENT_LIST.append(document)

### Pubmed Corpus from corpora.TextCorpus

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
#     def __init__(self, input=None, data_folder=DATA_PATH):
#         super(TextCorpus, self).__init__()
#         self.input = None
#         self.data_folder = os.walk(data_folder)
#         self.dictionary = Dictionary()
#         self.metadata = False        
#         if data_folder is not None:
#             self.dictionary.add_documents
    
#     def __iter__(self):
        
        
#     def get_texts(self, file_name):
#         with open(file_name, 'rb') as doc_file:
#             doc = doc_file.read()
#             doc_tokenized = utils.tokenize(doc, lowercase=True)
#             doc_tokenized = [word for word in doc_tokenized if word not in STOP_WORDS]
#             yield doc_tokenized

                    
# #         for doc_file in self.input:
# #             with open(doc_file, 'rb') as doc:
# #                 doc_text = doc.read()
# #                 tokenized_text = [token for token in utils.tokenize(doc_text, 
# #                                                                     lowercase=True)]
# #                 tokenized_text = [word for word in tokenized_text if word not in STOP_WORDS]
# #                 self.dictionary.add_documents([tokenized_text])
# #                 yield self.dictionary.doc2bow(tokenized_text, allow_update=False)
                
        
        
        
        
# #         directory = os.walk(self.input)
# #         for root, dirs, files in directory:
# #             for file_name in files:
# #                 file_path = os.path.join(root, file_name)
# #                 with open(file_path, 'rb') as f:
# #                     file_string = to_unicode_or_bust(f.read().lower())
# #                     file_tokenized = word_tokenize(file_string)
# #                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
# #                     yield file_no_stops